This is the template for DS3000 Final data analysis project. Once you finish, please remove all my instructions. You do not need to exactly follow the structure in the template but please make sure you have all the components. Write your report in paragraphs. Only use bullet points when list something (eg: functions) 

# Title 
#### Team number
- Samuel Hirn
- Sean O'Donnell
- Stephen Sodipo

In [1]:
# Put all the module import in this code chunk
# webscraping
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
# visualization


## Introduction

- One or two paragraphs about the background of the project. eg: the backgound of PalWorld and why your analysis can be interesting
- State your research questions. Limit the number of research questions to be one or two.

----

Palworld is a unique survival crafting game that combines creature collection mechanics with base building and combat elements. In Palworld, players can capture, train, and utilize various Pals, each with distinct stats and abilities that affect their performance in different tasks such as combat, resource gathering, and base management. The game's economy includes a pricing system for Pals, which creates an interesting opportunity to analyze what factors drive their market value.

Understanding the relationship between Pal characteristics and their prices has practical implications for players looking to optimize their gameplay strategies. By analyzing data from palworld.gg, a database of all Pals and their attributes, we can investigate which statistical features most strongly influence a Pal's price. Our primary research question is: **What combination of stats, elemental types, and rarity levels best predicts a Pal's price in the game?**

This analysis could help players make informed decisions about which Pals to prioritize for capture or breeding, and provide insights into the game's economic balancing mechanics.

-----

## Data 

### Data Source

- List the website you have scraped the data from.
- List which information you have scraped
- Describe what kind of cleaning you have done to the data

-----

**Website Scraped:**

Main Pal listing page (https://palword.gg/pals) and individual Pal detail pages (https://palworld.gg/pal/[pal_name])

**Information Scraped:**

The following attributes were extracted for each Pal:

Name, Rarity, Elements (can have multiple), HP, Defense, Melee Attack, Shot Attack, Crafting Speed, Support, Stamina, Running Speed, Sprinting Speed, Slow Walk Speed, Price, and a direct link to each Pal's detail page

**Data Cleaning Process:**

Several cleaning steps were implemented during the scraping process:

- Type Conversion: Converted numeric stat values from strings to integers where applicable, keeping non-numeric values as strings
- Missing Value Handling: Initialized all stat fields with None values to handle cases where certain stats might be missing from a Pal's page
- Element Formatting: Converted the list of elements for each Pal into a comma-separated string format for easier DataFrame storage
- Data Flattening: Restructured the nested dictionary format (with separate Stats and Elements dictionaries) into a flat structure suitable for pandas DataFrame creation
- Error Handling: Implemented try-except blocks to skip Pals that couldn't be properly scraped, preventing the entire process from failing due to individual page issues
- Standardized Naming: Converted stat names to a consistent format (lowercase with underscores) for uniform column naming in the final DataFrame

-----

### Webscraping and cleaning functions overview

List all the functions you have written for webscraping and data cleaning. For each one, write one sentence to describe it. 
- `extract_soup()`
    - build url and return soup object
- `fetch_pal(url: str) -> BeautifulSoup`
    - Fetches HTML content from a given URL and returns a parsed BeautifulSoup object.
- `parse_pals_data(main_soup: BeautifulSoup) -> list[dict]`
    - Extracts name and detail page links for all Pals from the main listing page.
- `strip_pal_stats(soup: BeautifulSoup) -> dict`
    - Scrapes individual Pal statistics including name, rarity, elements, and various stat values from a Pal's detail page.
- `create_dataframe() -> pd.DataFrame`
    - Orchestrates the entire scraping process by fetching all Pals, extracting their stats, and organizing the data into a pandas DataFrame.

## Webscraping and cleaning

In [2]:
# list all the functions you have for webscraping and cleaning. Make sure to write full docstrings for each function

def fetch_pal(url: str) -> BeautifulSoup:
    """
    Args:
        url (str): The inputted url link for a chosen pal

    Returns:
        BeatifulSoup: the parsed content for the given pal
    """
    # first get the link
    resp = requests.get(url)
    # have a checker in the event the input is wrong
    resp.raise_for_status()
    resp = resp.text
    return BeautifulSoup(resp, features="lxml")


def parse_pals_data(main_soup: BeautifulSoup) -> list[dict]:
    """
    Parses Pal data from palworld.gg HTML using BeautifulSoup,
    extracting Name and link to each Pal's detail page.

    Args:
        main_soup (BeautifulSoup): HTML content of the page.

    Returns:
        list[dict]: List of pals with 'Name' and 'Link'.
    """
    pals = []

    # Get the main section
    pals_section = main_soup.find('section', class_='pals-list')
    if not pals_section:
        print("No pals-list section found.")
        return []

    # Each Pal is a div with class "pal"
    pal_divs = pals_section.find_all('div', class_='pal')

    for pal_div in pal_divs:
        try:
            a_tag = pal_div.find('a', href=True)
            if not a_tag:
                continue

            detail_url = a_tag['href']
            full_url = "https://palworld.gg" + detail_url

            name_div = a_tag.find('div', class_='name')
            name = name_div.get_text(strip=True) if name_div else "Unknown"

            pals.append({"Name": name, "Link": full_url})

        except Exception as e:
            print(f"Error parsing pal card: {e}")
            continue

    return pals


def strip_pal_stats(soup: BeautifulSoup) -> dict:
    """
    Extracts individual stats from a Pal's page.

    Args:
        soup(BeautifulSoup): Parsed data from the Pal's page

    Return:
        dict: Dictionary of the stats
    """

    # Retrieve the name of the Pal
    name = soup.find('h1', class_='name').text.strip() if soup.find('h1', class_='name') else None

    # extract the rarity
    rarity_div = soup.find('div', class_='rarity')
    rarity = rarity_div.text.strip() if rarity_div else None

    # add proper space for rarity 
    if rarity:
        rarity = re.sub(r'(\d+)([A-Za-z]+)', r'\1 \2', rarity)
        
    print(rarity) # should give the space

    # Element name
    elements = []
    elements_div = soup.find('div', class_='elements')
    if elements_div:
        elements_divs = elements_div.find_all('div', class_='element')
        for elem_div in elements_divs:
            # this should get the element name
            elem_name = elem_div.find('div', class_ = 'name')
            if elem_name:
                elements.append(elem_name.text.strip())

    # stats
    # Initialize stats dictionary with expected fields
    stats = {
        'name': name,
        'rarity': rarity,
        'hp': None,
        'defense': None,
        'crafting_speed': None,
        'melee_attack': None,
        'shot_attack': None,
        'price': None,
        'stamina': None,
        'support': None,
        'running_speed': None,
        'sprinting_speed': None,
        'slow_walk_speed': None
    }

    stats_div = soup.find('div', class_='stats')
    # reach the stats section
    if stats_div:
        items_divs = stats_div.find_all('div', class_='item')
        # go into each item
        for item in items_divs:
            stat_name_div = item.find('div', class_='name')
            stat_value_div = item.find('div', class_='value')

            # using the name and value of the stats add to the dictionary
            if stat_name_div and stat_value_div:
                stat_name = stat_name_div.text.strip().lower().replace(' ', '_')
                stat_value = stat_value_div.text.strip()
                try:
                    stat_value = int(stat_value)
                except ValueError:
                    pass  # Keep as string if not an int


                if stat_name in stats:
                    stats[stat_name] = stat_value

    return {
        "Name": stats['name'],
        "Rarity": stats['rarity'],
        "Elements": elements,
        "Stats": {
            "HP": stats['hp'],
            "Defense": stats['defense'],
            "Crafting Speed": stats['crafting_speed'],
            "Melee Attack": stats['melee_attack'],
            "Shot Attack": stats['shot_attack'],
            "Support": stats['support'],
            "Stamina": stats['stamina'],
            "Price": stats['price'],
            "Running Speed": stats['running_speed'],
            "Sprinting Speed": stats['sprinting_speed'],
            "Slow Walk Speed": stats['slow_walk_speed'],
        }
    }


def create_dataframe() -> pd.DataFrame:
    """
    Create a DataFrame from the list of Pal data

    Returns:
        DataFrame: Cleaned Dataframe ready for analysis
    """

    main_pal_soup = fetch_pal(BASE_URL)

    # get the basic information
    pals_info = parse_pals_data(main_pal_soup)

    full_data = []
    # loop through the pals
    for idfx, pal in enumerate(pals_info):
        try:
            print(f"Processing {idfx+1}/{len(pals_info)}: {pal['Name']}")
            detail_soup = fetch_pal(pal["Link"])
            stats = strip_pal_stats(detail_soup)

            # flatten the stats dictionary
            rec = {
                'Name': stats.get('Name'),
                "Link": pal.get("Link"),
                "Rarity": stats['Rarity']
            }

            # add stats (HP, Defense, etc.)
            for key, value in stats.get("Stats", {}).items():
                rec[key] = value
            # add elements as a joined string
            rec["Elements"] = ", ".join(stats.get("Elements", []))

            full_data.append(rec)
        except Exception as e:
            print(f"Skipping {pal.get('Name', '')}: {e}")
            continue

    df = pd.DataFrame(full_data)
    return df

In [3]:
# Write the code to load the data with the functions. You don't need to run the code every time. 
# You can run the code once and save the scrapped data into a csv file. Then load the csv file 
# for the rest of the analysis
BASE_URL = "https://palworld.gg/pals"

df = create_dataframe()
df.to_csv("test_data1.csv", index=False)
df.head(10)

Processing 1/225: Anubis#100
10 Epic
Processing 2/225: Arsox#42
4 Common
Processing 3/225: Astegon#98
9 Epic
Processing 4/225: Azurmane#136
7 Rare
Processing 5/225: Azurobe#82
7 Rare
Processing 6/225: Azurobe Cryst#82
8 Epic
Processing 7/225: Bastigor#137
8 Epic
Processing 8/225: Beakon#73
6 Rare
Processing 9/225: Beegarde#50
4 Common
Processing 10/225: Bellanoir#112
20 Legendary
Processing 11/225: Bellanoir Libero#112
20 Legendary
Processing 12/225: Blazamut#96
9 Epic
Processing 13/225: Blazamut Ryu#96
10 Epic
Processing 14/225: Blazehowl#84
7 Rare
Processing 15/225: Blazehowl Noct#84
8 Epic
Processing 16/225: Blue Slime#-1
3 Common
Processing 17/225: Braloha#145
6 Rare
Processing 18/225: Bristla#30
1 Common
Processing 19/225: Broncherry#86
7 Rare
Processing 20/225: Broncherry Aqua#86
8 Epic
Processing 21/225: Bushi#72
7 Rare
Processing 22/225: Bushi Noct#72
7 Rare
Processing 23/225: Caprity#35
3 Common
Processing 24/225: Caprity Noct#35
3 Common
Processing 25/225: Cattiva#2
1 Common


,Name,Link,Rarity,HP,Defense,Crafting Speed,Melee Attack,Shot Attack,Support,Stamina,Price,Running Speed,Sprinting Speed,Slow Walk Speed,Elements
0,Anubis,https://palworld.gg/pal/anubis,10 Epic,120,100,100,130,130,100,100,4960,800,1000,80,Earth
1,Arsox,https://palworld.gg/pal/arsox,4 Common,85,95,100,100,95,100,120,3520,700,1050,87,Fire
2,Astegon,https://palworld.gg/pal/astegon,9 Epic,100,125,100,100,125,100,300,8200,700,1100,100,"Dragon, Dark"
3,Azurmane,https://palworld.gg/pal/azurmane,7 Rare,130,110,100,100,120,100,220,6680,900,1260,90,Electricity
4,Azurobe,https://palworld.gg/pal/azurobe,7 Rare,110,100,100,70,100,100,160,5600,600,900,75,"Water, Dragon"
5,Azurobe Cryst,https://palworld.gg/pal/azurobe-cryst,8 Epic,115,105,100,100,105,100,160,6720,600,900,75,"Ice, Dragon"
6,Bastigor,https://palworld.gg/pal/bastigor,8 Epic,140,120,100,100,130,100,270,9020,750,1100,120,Ice
7,Beakon,https://palworld.gg/pal/beakon,6 Rare,105,80,100,100,115,100,160,7490,750,1200,100,Electricity
8,Beegarde,https://palworld.gg/pal/beegarde,4 Common,80,90,100,100,90,100,100,1880,450,550,125,Leaf
9,Bellanoir,https://palworld.gg/pal/bellanoir,20 Legendary,120,100,100,100,150,100,100,10030,600,800,100,Dark


### Data overview

- Show a couple of rows of the cleaned data you are going to use for the analysis
- Which is your target value (if exists)
- Give a general summary about the other features
- Discuss if there is any potential problems about the data (eg: missing values, any features that you did not collect but may be important, any other concerns)

-----
**Sample Data:**

Cleaned data preview is shown above.

**Target Variable:**

Price is our target variable, ranging from 1,000 to 24,900 (mean: 4,535)
All 225 Pals have price values (no missing data)

**Feature Summary:**

Combat Stats: HP (60-150), Defense (50-145), Melee Attack (50-150), Shot Attack (50-150)
Utility Stats: All Pals have Crafting Speed of 100, Support mostly at 100 (range: 30-150)
Movement Stats: Running Speed (240-2000), Sprinting Speed has 2 anomalous negative values
Categorical: Rarity levels from 1Common to 20Legendary; 9 element types with 72% of Pals having single element

**Potential Data Issues:**

- No data on skills/abilities, work suitability, food consumption, or capture difficulty - which could significantly impact price
- 2 Pals have negative Sprinting Speed values (-1), suggesting data collection errors
- Crafting Speed is constant at 100 for all Pals, providing no predictive value
- Only 8 Legendary Pals vs 30+ Common ones could affect model performance

-----

## Visualizations

### Visualization functions overview
List all the functions you have written for visualization. For each one, write one sentence to describe it. 
- `make_hist()`
    - Generate a histogram with given data and feature
 
### Visualization results
- Present 3-4 data visualizations.
- For each visualization, you need to include title, xlabel, ylabel, legend (if necessary)
- For each visualization, explain why you make this data visualization (how it related to your research question) and explain what you have learned from this visualization

In [ ]:
# list all the functions you have for visualization. Make sure write full 
# docstrings for each function
def make_hist(df, y_feat):

    pass

#### visualization 1

In [ ]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

#### visualization 2

In [ ]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

#### visualization 3

In [ ]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

## Models

### Modeling functions overview
List all the functions you have written for modeling. For each one, write one sentence to describe it. 
- `fit_linear()`
    - fit a linear model to the data and output the r2, slope and intercept

### Model results

- Present 2-3 models for the analysis.
- Explain any pre-processing steps you have done (eg: scaling, polynomial, dummy features)
- For each model, explain why you think this model is suitable and what metrics you want to use to evaluate the model
    - If it is a classification model, you need to present the confusion matrix, calculate the accuracy, sensitivity and specificity with cross-validation
    - If it is a regression model, you need to present the r2 and MSE with cross-validation
    - If it is a linear regression model/multiple linear regression model, you need to interpret the meaning of the coefficient with the full data
    - If it is a decision tree model, you need to plot the tree with the full data
    - If it is a random forest model, you need to present the feature importance plot with the full data
    - If it is a PCA, you need to explain how to select the number of components and interpret the key features in the first two components
    - If it is a clustering, you need explain how to select the number of clustering and summarize the clustering. 

In [ ]:
# list all the functions you have for modeling. Make sure write full 
# docstrings for each function
def fit_linear(df, y_feat, x_feat):

    pass

#### Model 1

In [ ]:
# Write the code to run functions to fit each model in separate code chunks. 
# Interpret the model results. 

#### Model 2

In [ ]:
# Write the code to run functions to fit each model in separate code chunks. 
# Interpret the model results. 

## Discussion

- One or two paragraphs to summarize your findings in the modeling sections and do the models answer your research question?
- Any other potential thing you can do with the analysis (eg: include more features, get more data, try some other models etc.)
- List the contribution for each group member.